In [1]:
from agent.agents.embedding_model import EmbeddingModel
from agent.models.input import EmbeddingModelInput
from agent.models.context import Context
from agent.models.issue import Issue

from IPython.display import clear_output
import json

/Users/ikerochoa/opt/anaconda3/envs/kaizen-copilot-py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from agent.agents.query_formatter import QueryFormatter
from agent.models.input import QueryFormatterInput


filepath = "./eval/swe_bench/data/swe_bench_lite_test.jsonl"

correct_files = 0
not_found_files = []

with open(filepath, "r", encoding="utf-8") as f:
    for idx, line in enumerate(f):
        if line.strip():  # skip empty lines
            if idx % 3 == 0: clear_output(wait=True)
            
            obj = json.loads(line)
            
            issue = Issue("", obj["problem_statement"], f"{obj['repo']}")
            context = Context("issue_resolution", issue=issue)
            
            input_data = QueryFormatterInput(issue=issue)
            
            input_data, context, _ = QueryFormatter().run(input_data, context)
            
            input_data =  EmbeddingModelInput(queries=input_data.queries, conditionals=input_data.conditionals)
            
            _, files_result, _ = EmbeddingModel().run(input_data, context)   
            result_file_name = files_result.retrieved_files[0].name
            is_correct = result_file_name in obj["patch"]
            
            total = idx + 1 
            file_index = str(total)
            if is_correct:
                print("File found: " + file_index)
            else:
                print("File not found: " + file_index)
                not_found_files.append(file_index)
                
            correct_files += 1 if is_correct else 0
            if correct_files != 0: 
                current_media =  (correct_files/total) * 100
                print("Current Media: " + str(current_media))

File not found: 298
Current Media: 22.14765100671141
File not found: 299
Current Media: 22.073578595317723
File found: 300
Current Media: 22.333333333333332


In [ ]:
total_media = (correct_files / 300) * 100
print("File Query Accuracy: " + str(total_media))

File Query Accuracy: 22.666666666666664


In [8]:
not_found_files[:5]

[3, 5, 6, 7, 10]